<a href="https://colab.research.google.com/github/HiskeOverweg/bo_intro/blob/master/bo_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Bayesian optimization

You can run a cell by clicking on it and pressing shift+Enter. The cell below will install some required packages

In [0]:
!pip install git+https://github.com/HiskeOverweg/bo_intro.git --upgrade
!pip install botorch

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import botorch
import gpytorch
import bo_intro.datasets
from bo_intro.run_bayesian_optimization import run_bo_experiment

##Finding the maximum of the sine function on the interval [0, 2$\pi$]

We can run Bayesian optimization with 1 random starting point and 20 iterations on the sine function as follows:

In [0]:
config =  {'iterations':20, 'initial_observations':1, 'dataset':'sine', 'acquisition_function':'ei', 'noise':0}
x, y = run_bo_experiment(config, print_progress=True, seed=0)

**Exercise 1** Plot a sine function and the datapoints x, y queried by the Bayesian optimization algorithm

Let's fit a Gaussian process to the complete dataset. We can plot its mean and the confidence bound (2 standard deviations away from the mean).

In [0]:
def plot_gaussian_process(x, y):
  dataset = bo_intro.datasets.Sine()
  x_scaled = dataset.scale(torch.from_numpy(x))

  gaussian_process = botorch.models.SingleTaskGP(x_scaled, torch.from_numpy(y))
  mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood=gaussian_process.likelihood, model=gaussian_process)
  botorch.fit.fit_gpytorch_model(mll)

  x_test = torch.linspace(0, 1, 20, dtype=torch.double).unsqueeze(dim=1)
  posterior = gaussian_process.posterior(x_test)
  lower, upper = posterior.mvn.confidence_region()

  plt.plot(dataset.rescale(x_test), posterior.mean.detach())
  plt.plot(x, y, 'o')
  plt.fill_between(dataset.rescale(x_test).squeeze(), lower.detach(), upper.detach(), alpha=0.5);
  plt.xlim([0, 2*np.pi]);

plot_gaussian_process(x, y)

**Exercise 2** Do you understand the shape of the confidence bound?

**Exercise 3** Try adding some noise to the observations by adapting the 'noise' value in the config dictionary. The corresponding value is the standard deviation of the Gaussian distributed noise. Plot the obtained x and y values. Is the position of the maximum close to the expected maximum at $\pi$/2?

##Regret

The regret is defined as the difference between the true maximum of the function and the best value found so far.

**Exercise 4** Plot the regret as a function of iteration number, using a logarithmic y-axis

In [0]:
running_max = np.maximum.accumulate(y)

Since Bayesian optimization is a stochastic algorithm it can be useful to evaluate the regret over a few different initializations of the algorithm.

**Exercise 5** Run the algorithm 5 times with different random seeds and make a plot of the regret as a function of iteration number

##Comparing acquisition functions

Let us now compare a few acquisition functions. You can specify the key 'acquisition_function' in the config dictionary to switch to 'random' or 'ucb' (Upper Confidence Bound).

**Exercise 6** Repeat exercise 5 with a random acquisition function. Which acquisition function leads to the lowest regret?

##Exploring vs exploiting
The upper confidence bound acquisition function is defined as $\mu + \beta \sigma$, where $\mu$ and $\sigma$ are the mean and standard deviation of the Gaussian process and $\beta$ is a constant. By increasing $\beta$ we can make the search more explorative. The default value is $\beta = 3$, but you can change it by specifying for instance 'beta':6 in the config dictionary.

**Exercise 7** Plot a sine function and the datapoints x, y queried by the Bayesian optimization algorithm with ucb acquisition function and 'beta':6.

## Optimizing a 2-dimensional function

**Exercise 8** Try optimizing the [negative Branin function](https://www.sfu.ca/~ssurjano/branin.html) by specifying 'dataset':'branin' in the config dictionary. Make a plot of regret vs iteration number (the maximum value of the negative branin function is at -0.397887).

In [0]:
true_max = -0.397887

## Template for optimization of measurement in Labber

A dataset which would perform an experiment in Labber to acquire new datapoints would roughly look like (see also [Labber documentation about scripting](http://labber.org/online-doc/api/ScriptTools.html/)): 

In [0]:
import os
from Labber import ScriptTools

class LabberExperiment(Sine):
    def __init__(self, config={}):
        bounds = torch.tensor([[0, 1]], dtype=torch.double)
        self.min, _ = torch.min(bounds, dim=1, keepdim=True)
        self.min = torch.transpose(self.min, 0, 1)
        self.interval = torch.abs(bounds[:, 0] - bounds[:, 1])
        self.dim = bounds.shape[0]
        self.num_points = config.setdefault('initial_observations', 0)
        self.x = torch.rand(self.num_points, self.dim, dtype=torch.double)
        self.y = self.query(self.x)
        # define measurement objects
        sPath = os.path.dirname(os.path.abspath(__file__))
        self.MeasResonator = ScriptTools.MeasurementObject(\
                os.path.join(sPath, 'TestResonator.hdf5'),
                os.path.join(sPath, 'TestResonatorOut.hdf5'))
        self.MeasResonator.setMasterChannel('Flux bias')

    def query(self, x):
        x_rescaled = self.rescale(x)
        results = []
        for setting in x:
          self.MeasResonator.updateValue('Flux bias', setting.numpy())
          (x,y) = self.MeasResonator.performMeasurement()
          results.append(y)
        return torch.tensor(results, dtype=torch.double).unsqueeze(dim=1)

##Batch mode Bayesian optimization

It is also possible to run Bayesian optimization in batch mode: rather than querying for the next most informative datapoint, we can ask for a batch of N most informative datapoints. This can be especially useful in a simulation, where you can evaluate multiple settings in parallel. More details about batch mode can be found [here](https://botorch.org/docs/batching#docsNav).

##Conclusion

I hope this intro helped to get a basic understanding of Bayesian optimization. If you come up with a way to use it in your own experiments, please let me know, I am curious to hear about it!